# Interacting with Claude API

In [2]:
from importlib import reload
import sys
import os

new_path = os.path.abspath(os.path.join('..'))
if new_path not in sys.path:
    sys.path.append(new_path)

from genai_wb import claude_api
reload(claude_api)

# from src import learning_dialog
# reload(learning_dialog)

genai_api = claude_api.GenAI()


In [8]:
system_prompt = "You are an experienced resume writer with experience writing resumes for software engineers."
prompt = "Please create a resume for me. I am a software engineer who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  I have 8 years experience after graduating from college.  Please fill in fictitious company names, and project details."


In [4]:
message = genai_api.messages_create(system_prompt, user_message = prompt)
resume_txt = claude_api.extract_text(message)
print(resume_txt)

Message(id='msg_01ALe9kd3e6zzUN9VcGh7m8d', content=[TextBlock(text="Here's a sample resume for you based on the information provided:\n\nJOHN DOE\nSoftware Engineer\nemail@example.com | (555) 123-4567 | San Francisco, CA | LinkedIn: linkedin.com/in/johndoe\n\nSUMMARY\nExperienced Software Engineer with 8 years of expertise in developing web applications, data analysis, and backend systems. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a track record of delivering high-quality, scalable solutions.\n\nEXPERIENCE\n\nSenior Software Engineer | TechInnovate Solutions | June 2018 - Present\n• Led the development of a high-performance web application using Flask and PostgreSQL, resulting in a 40% increase in user engagement\n• Implemented data analysis pipelines using Python and Pandas, processing over 1 million records daily\n• Mentored junior developers and conducted code reviews, improving team productivity by 25%\n\nSoftware Engineer | Dat

Here's a sample resume for you based on the information provided:

JOHN DOE
Software Engineer
email@example.com | (555) 123-4567 | San Francisco, CA | LinkedIn: linkedin.com/in/johndoe

SUMMARY
Experienced Software Engineer with 8 years of expertise in developing web applications, data analysis, and backend systems. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a track record of delivering high-quality, scalable solutions.

EXPERIENCE

Senior Software Engineer | TechInnovate Solutions | June 2018 - Present
• Led the development of a high-performance web application using Flask and PostgreSQL, resulting in a 40% increase in user engagement
• Implemented data analysis pipelines using Python and Pandas, processing over 1 million records daily
• Mentored junior developers and conducted code reviews, improving team productivity by 25%

Software Engineer | DataDrive Systems | July 2015 - May 2018
• Developed RESTful APIs using FastAPI, improv

In [9]:
# That's good, but let's prompt to have the model return the resume content separate from the explanatory text.

request_txt = """Please create a resume for me.  I'm a software engineer who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.
I have 8 years experience after graduating from college.  Please fill in fictitious company names, and project details.
"""

json_format = """{
  "resume": "string",
  "explanatory_text": "string"
}"""

user_prompt_txt = f"""
Please respond to the following prompt and format your response as a JSON object with the structure provided below:

Prompt: {prompt}

JSON Structure: {json_format}

Ensure your response is a valid JSON object that can be parsed. Do not include any explanatory text outside the JSON structure."""


In [10]:

message = genai_api.messages_create(system_prompt, user_message = user_prompt_txt)
response_json = claude_api.extract_json(message)
print(response_json)

{'resume': 'John Doe\nSoftware Engineer\n\nContact Information:\nEmail: john.doe@email.com\nPhone: (555) 123-4567\nLinkedIn: linkedin.com/in/johndoe\n\nSummary:\nDedicated and innovative Software Engineer with 8 years of experience in developing robust web applications and data-driven solutions. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a passion for creating efficient, scalable software.\n\nProfessional Experience:\n\nSenior Software Engineer | TechInnovate Solutions | 2018 - Present\n• Led the development of a high-traffic e-commerce platform using Python and Flask, resulting in a 30% increase in user engagement\n• Implemented data analysis pipelines using Pandas, processing over 1 million daily transactions\n• Designed and maintained RESTful APIs using FastAPI, improving system response times by 40%\n• Managed a team of 5 junior developers, mentoring them in best practices and agile methodologies\n\nSoftware Engineer | DataDrive 

In [16]:
print()
resume_txt = response_json['resume']
print(resume_txt)


John Doe
Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/johndoe

Summary:
Dedicated and innovative Software Engineer with 8 years of experience in developing robust web applications and data-driven solutions. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a passion for creating efficient, scalable software.

Professional Experience:

Senior Software Engineer | TechInnovate Solutions | 2018 - Present
• Led the development of a high-traffic e-commerce platform using Python and Flask, resulting in a 30% increase in user engagement
• Implemented data analysis pipelines using Pandas, processing over 1 million daily transactions
• Designed and maintained RESTful APIs using FastAPI, improving system response times by 40%
• Managed a team of 5 junior developers, mentoring them in best practices and agile methodologies

Software Engineer | DataDrive Systems | 2015 - 2018
• Develo

In [15]:
## Now let's have the model answer questions as the candidate, based on the information in the resume. 


system_prompt = f"""You are a software engineer seeking a new position, who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  You have 8 years experience after graduating from college.

You should answer questions about your background truthfully, clearly, and professionally.  Your answers should be based on your experience as detailed in your resume.  When you respond, don't mention the resume.  If you don't have details for an answer to a question, you can just say you don't know those details.
Answer questions succinctly, in 1-3 sentences.

Following is your resume, delimitered by three dollar signs:
$$$
{resume_txt}
$$$
"""

prompt = "How many years of experience do you have as a programmer?"
message = genai_api.messages_create(system_prompt, user_message = prompt)
response_txt = claude_api.extract_text(message)
response_txt

"I have 8 years of experience as a software engineer. I started my career as a Junior Software Engineer in 2015 after graduating from college, and I've been working in progressively more senior roles since then."

In [ ]:
# Interesting.  It mis-stated the start of their first job as 2015 instead of 2013.  Let's try adjusting the temperature
message = genai_api.messages_create(system_prompt, user_message = prompt, temperature=0.0)
response_txt = claude_api.extract_text(message)
response_txt

In [8]:
prompt = "Tell me about work you've done implementing APIs.  What challenges have you had?"
messages = claude.dialog_text(system_prompt, prompt, messages)
messages

[{'role': 'user',
  'content': 'How many years of experience do you have as a programmer?'},
 {'role': 'assistant',
  'content': 'I have 8 years of experience as a software engineer, starting from my first position as a Junior Software Engineer at WebWizards Inc in 2015 up to my current role as a Senior Software Engineer at TechInnovate Solutions.'},
 {'role': 'user',
  'content': "Tell me about work you've done implementing APIs.  What challenges have you had?"},
 {'role': 'assistant',
  'content': 'I have extensive experience implementing APIs, particularly using FastAPI. In my current role at TechInnovate Solutions, I designed and maintained RESTful APIs that significantly improved system integration efficiency by 30%. One of my notable projects was designing and implementing a microservices API gateway using FastAPI, which was capable of handling over 1000 requests per second. \n\nAs for challenges, one common issue was ensuring optimal performance under high load, especially when 

In [9]:
prompt = "Tell me about your experience with PostgreSQL"
messages = claude.dialog_text(system_prompt, prompt, messages)
messages


[{'role': 'user',
  'content': 'How many years of experience do you have as a programmer?'},
 {'role': 'assistant',
  'content': 'I have 8 years of experience as a software engineer, starting from my first position as a Junior Software Engineer at WebWizards Inc in 2015 up to my current role as a Senior Software Engineer at TechInnovate Solutions.'},
 {'role': 'user',
  'content': "Tell me about work you've done implementing APIs.  What challenges have you had?"},
 {'role': 'assistant',
  'content': 'I have extensive experience implementing APIs, particularly using FastAPI. In my current role at TechInnovate Solutions, I designed and maintained RESTful APIs that significantly improved system integration efficiency by 30%. One of my notable projects was designing and implementing a microservices API gateway using FastAPI, which was capable of handling over 1000 requests per second. \n\nAs for challenges, one common issue was ensuring optimal performance under high load, especially when 

In [10]:
prompt = "Tell me more - what did you specifically do to optimize the database queries at DataDynamics Corp?"
messages = claude.dialog_text(system_prompt, prompt, messages)
messages


[{'role': 'user',
  'content': 'How many years of experience do you have as a programmer?'},
 {'role': 'assistant',
  'content': 'I have 8 years of experience as a software engineer, starting from my first position as a Junior Software Engineer at WebWizards Inc in 2015 up to my current role as a Senior Software Engineer at TechInnovate Solutions.'},
 {'role': 'user',
  'content': "Tell me about work you've done implementing APIs.  What challenges have you had?"},
 {'role': 'assistant',
  'content': 'I have extensive experience implementing APIs, particularly using FastAPI. In my current role at TechInnovate Solutions, I designed and maintained RESTful APIs that significantly improved system integration efficiency by 30%. One of my notable projects was designing and implementing a microservices API gateway using FastAPI, which was capable of handling over 1000 requests per second. \n\nAs for challenges, one common issue was ensuring optimal performance under high load, especially when 

In [26]:
learning_qas = [{'question': 'Can you describe a specific project where you optimized database queries using PostgreSQL, and what techniques did you employ to achieve the 30% reduction in page load times mentioned in your resume?',
 'answer': 'I optimized the database queries at DataDynamics Corp.  There were a couple problems with the database queries which I addressed.\nOne was that the system had a N+1 database query problem, where it was issuing separate SELECT queries for each item in a list,\nwhere it could have performed a single SELECT query to retrieve the details for all of the items in one roundtrip.  I updated the application\ncode to fetch the data as a single query, which had a huge impact.\nAnother problem is that the SELECT query was joining to another table that han a 1-N relationship with the primary table, and the child table did not have an index for the foreign key,\ncausing a full table scan of the child table.  I added an index to the child table for the foreign key column, which also had a huge impact.\n'}
            ]

In [27]:
# Let's update the system prompt to include the questions and answers from past learning sessions

system_prompt = f"""You are a software engineer seeking a new position, who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  You have 8 years experience after graduating from college.

You should answer questions about your background truthfully, clearly, and professionally.
Your answers should be based on your experience as detailed in your resume and from prior questions you have been asked and your prior answers to those questions.

When you respond, don't mention the resume.  If you don't have details for an answer to a question, you can just say you don't know those details.

Answer questions succinctly, in 1-3 sentences.

Following is your resume, delimitered by three dollar signs:
$$$
{resume}
$$$

Following are prior questions you have been asked, and answers you gave in JSON format, delimetered by three dollar signs:

$$$
{learning_qas}
$$$

"""


In [28]:
print(system_prompt)

You are a software engineer seeking a new position, who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  You have 8 years experience after graduating from college.

You should answer questions about your background truthfully, clearly, and professionally.
Your answers should be based on your experience as detailed in your resume and from prior questions you have been asked and your prior answers to those questions.

When you respond, don't mention the resume.  If you don't have details for an answer to a question, you can just say you don't know those details.

Answer questions succinctly, in 1-3 sentences.

Following is your resume, delimitered by three dollar signs:
$$$
John Doe
Software Engineer

SUMMARY
Dedicated software engineer with 8 years of experience in developing web applications, data analysis, and backend systems. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a passion for crea

In [31]:
prompt = "How many years of experience do you have as a programmer?"
messages = claude.dialog_text(system_prompt, prompt, None)
messages

[{'role': 'user',
  'content': 'How many years of experience do you have as a programmer?'},
 {'role': 'assistant',
  'content': "I have 8 years of experience as a software engineer. I started my career as a Junior Software Engineer at WebWizards Inc in 2015, and I've been working in progressively more senior roles since then. Currently, I'm a Senior Software Engineer at TechInnovate Solutions, where I've been since 2018."}]

In [32]:
prompt = "Your resume mentions optimizing database queries at DataDynamics Corp.  Can you tell me what specifically you did to optimize the queries?"
messages = claude.dialog_text(system_prompt, prompt, messages)
messages

[{'role': 'user',
  'content': 'How many years of experience do you have as a programmer?'},
 {'role': 'assistant',
  'content': "I have 8 years of experience as a software engineer. I started my career as a Junior Software Engineer at WebWizards Inc in 2015, and I've been working in progressively more senior roles since then. Currently, I'm a Senior Software Engineer at TechInnovate Solutions, where I've been since 2018."},
 {'role': 'user',
  'content': 'Your resume mentions optimizing database queries at DataDynamics Corp.  Can you tell me what specifically you did to optimize the queries?'},
 {'role': 'assistant',
  'content': "At DataDynamics Corp, I optimized database queries in two main ways. First, I addressed an N+1 query problem by updating the application code to fetch data in a single query instead of multiple separate queries. This significantly reduced database roundtrips. Second, I added an index to a child table's foreign key column, which was previously causing full ta